# Продвинутый Python, ДЗ-1

Правила игры:

* В домашке 5 задач, разбалловка указана в названии задачи. Суммарно за дз можно получить 100 баллов, что равняется 10 баллам

* В каждой задаче необходимо реализовать функцию, которая после будет проверяться через github classroom на тестах. Сами тесты лежит в гитхабе, можете локально проверить работу функций перед сдачей

* Дедлайн - 7 дней после выдачи дз. Необходимо залить решеннный ноутбук в github и прислать ссылку в Anytask (без выполнения любого из пунктов работа проверяться не будет)

* В данной домашке нужно использовать numpy и scipy

In [1]:
import math
import time
import random
import warnings

import numpy as np
from scipy.stats import qmc
import matplotlib.pyplot as plt

from IPython.display import clear_output
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)

SEED = 21
random.seed(SEED)
np.random.seed(SEED)

In [2]:
# https://stackoverflow.com/a/60658965/7286121

from IPython.core.magic import register_cell_magic

@register_cell_magic
def write_and_run(line, cell):
    argz = line.split()
    file = argz[-1]
    mode = 'w'
    if len(argz) == 2 and argz[0] == '-a':
        mode = 'a'
    with open(file, mode) as f:
        f.write(cell)
    get_ipython().run_cell(cell)

# Задание №0 (10 баллов)

Реализуйте с помощью ```np.einsum``` следующие операции для векторов:

1. ```np.sum(A)```
2. ```A * B```
3. ```np.inner(A, B)```
4. ```np.outer(A, B)```


In [22]:
%%write_and_run einsum_task.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import random

SEED = 21
random.seed(SEED)
np.random.seed(SEED)


def task_00(A):
    if A.ndim > 1:
        result = np.einsum("ij->", A)
    else:
        result = np.einsum("i->", A)
    return result


def task_01(A, B):
    if A.ndim > 1:
        result = np.einsum("ij,ij->ij", A, B)
    else:
        result = np.einsum("i,i->i", A, B)
    return result


def task_02(A, B):
    if A.ndim > 1:
        result = np.einsum("ik,jk->", A, B)
    else:
        result = np.einsum("i,i->", A, B)
    return result


def task_03(A, B):
    result = np.einsum("i,j->ij", A, B)
    return result

# Задание №1 (10 баллов)

Фильтр Гаусса - это мощное средство при обработке изображений, которое позволяет избавиться от лишнего шума в картинке (а также дает возможность размыть картинку)

Для наглядности как работает фильтр:

![](https://upload.wikimedia.org/wikipedia/commons/thumb/6/62/Cappadocia_Gaussian_Blur.svg/800px-Cappadocia_Gaussian_Blur.svg.png)

Вам нужно реализовать с помощью numpy двухмерный фильтр Гаусса. Как он вычисляется:

$$G = e^{- \frac{(D - μ)^{2}}{2 \sigma^{2}} }$$

где

$$D = \sqrt{X^2 + Y^2} $$

а $\mu$ и $\sigma$ - это константы (среднее и стандартное отклонение)

Посчитайте фильтр Гаусса на координатах от -1 до 1 (10 точек по каждой оси, подсказка: используйте np.meshgrid)

In [21]:
%%write_and_run gauss_filter.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np


def gauss_filter(sigma=1.0, mu=0.0):
    x = np.linspace(-1, 1, 10)
    y = np.linspace(-1, 1, 10)
    xg, yg = np.meshgrid(x, y)
    xg = xg * xg
    yg = yg * yg
    answer = xg + yg

    do_gauss = np.vectorize(lambda t: np.exp(-((t - mu) ** 2) / (2 * (sigma ** 2))))
    answer = np.sqrt(answer)
    answer = do_gauss(answer)

    return answer

# Задание №2 (10 баллов)

Требуется найти k наибольших элементов в массиве с помощью numpy

In [20]:
%%write_and_run greatest_task.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import random

SEED = 21
random.seed(SEED)
np.random.seed(SEED)


def find_largest_element(array, n=7):
    new_array = np.copy(array)
    new_array = np.sort(new_array)[::-1]
    new_array = new_array[:n]
    return new_array[::-1]

# Задание №3 (20 баллов)

**Game of life**

Место действия игры — размеченная на клетки плоскость, которая может быть безграничной, ограниченной или замкнутой. Каждая клетка на этой поверхности имеет восемь соседей, окружающих её, и может находиться в двух состояниях: быть «живой» (заполненной) или «мёртвой» (пустой). Распределение живых клеток в начале игры называется первым поколением. Каждое следующее поколение рассчитывается на основе предыдущего по таким правилам:
* в пустой (мёртвой) клетке, с которой соседствуют три живые клетки, зарождается жизнь;
* если у живой клетки есть две или три живые соседки, то эта клетка продолжает жить; в противном случае (если живых соседей меньше двух или больше трёх) клетка умирает («от одиночества» или «от перенаселённости»).


Игра прекращается, если на поле не останется ни одной «живой» клетки; конфигурация на очередном шаге в точности (без сдвигов и поворотов) повторит себя же на одном из более ранних шагов (складывается периодическая конфигурация) при очередном шаге ни одна из клеток не меняет своего состояния (частный случай предыдущего правила, складывается стабильная конфигурация)

Ваша задача реализовать игру с помощью numpy

In [ ]:
%%write_and_run game_of_life.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import random

SEED=21
random.seed(SEED)
np.random.seed(SEED)

def count_alive(array, n, m, i, j):
    x = 0
    if i == 0 or i == n - 1 or j == 0 or j == m - 1:
        return 0
    else:
        x += array[i - 1, j - 1]
        x += array[i - 1, j]
        x += array[i - 1, (j + 1) % m]
        x += array[i, j - 1]
        x += array[i, (j + 1) % m]
        x += array[(i + 1) % n, j - 1]
        x += array[(i + 1) % n, j]
        x += array[(i + 1) % n, (j + 1) % m]

        if x == 3:
            return 1
        if x < 2 or x > 3:
            return 0
        return array[i][j]


def game_of_life_next_step(array):
    n, m = array.shape
    change_positions = np.array([count_alive(array, n, m, i, j) for i in range(n) for j in range(m)])
    change_positions = change_positions.reshape((n, m))

    return change_positions

## Задание №4 (50 баллов)

Вспомним алгоритм дифференциальной эволюции, который проходили на семинаре:

1. Инициализация популяции: Начните с создания случайной популяции, где каждый индивид (вектор) представляет собой набор параметров, которые нужно оптимизировать. Эти параметры могут быть представлены вещественными числами.

2. Оператор мутации: Для каждого индивида в популяции создается мутантный индивид. Это делается путем комбинирования параметров из нескольких случайно выбранных индивидов с использованием оператора мутации. Обычно оператор мутации может выглядеть так:
$mutant = a + mutcoeff * (b - c)$

3. Оператор кроссовера (рекомбинации): Мутантный индивид объединяется с исходным индивидом с использованием оператора кроссовера (рекомбинации). Этот оператор помогает определить, какие параметры оставить от исходного индивида и какие параметры заменить мутантными значениями.

4. Оценка потомка: Оцените качество потомка, полученного после кроссовера, с помощью целевой функции.

5. Селекция. Здесь происходит выбор индивида для следующего поколения: Сравните качество потомка и исходного индивида. Если потомок оказывается лучше, чем исходный индивид, то он становится частью следующего поколения. В противном случае, оставляем исходного индивида.

6. Повторение шагов 3-6: Повторяйте операции мутации, кроссовера, оценки и выбора индивидов для следующего поколения некоторое количество итераций или до достижения заданного критерия останова (например, максимальное число итераций или достижение заданной точности).

**Ваша задача:**
1. Реализовать инциализацию популяции с помощью различных распределений, таких как:

    a. [LatinHypercube](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.qmc.LatinHypercube.html)

    b. [Halton](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.qmc.Halton.html)

    c. [Sobol](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.qmc.Sobol.html)

2. Реализовать различные операторы мутации:

    a. *Rand2* - выбор 5 случайных элементов $mutant = a + mutcoeff * (b - c) + mutcoeff * (d - e)$

    b. *Best1* - использование лучшего элемента, исключая текущего лучшего и нынешний элемент итерации в мутации $mutant = best1 + mutcoeff * (b - c)$

    c. *Rand-to-p-best* - использование одного из $p$ процентов лучших элемента, исключая текущего лучшего и нынешний элемент итерации в мутации $mutant$ = *random better element* $+ mutcoeff * (b - c)$
3. Реализовать различные виды селекции

    Сейчас селекция происходит по потомку и исходному индвиду. В данном задании требуется реализовать сравнение не по исходному индвидиду, а по:
    
    a. По худшему индивиду в популяции индвидов (предков)

    b. Случайно выбирать индивид из худших индвидов по отношению к исходному. Если таковых нет, то выбирать исходный.

    c. Выбирать случайный индвид не включая лучший

**ВАЖНО**:
1. Везде использовать SEED, в том числе при инициализации начальных распределений
2. Вместо ```random.choice``` использовать ```np.random.choice```.
3. Не удалять строчку ```p = np.random.uniform(p_min, p_max)```.
4. После инициализации не забудьте размешать ```population.random(n=population_size)```.
5. Постарайтесь не использовать циклы.
6. Для инициализации Соболя попробуйте использовать ```random_base2``` и ```np.log2```, если не удовлетворяет, то как обычно.
7. Попробуйте использовать ```np.setdiff1d(..., assume_unique=True)```
8. Вместо ```min```/```max``` используйте ```np.min```, ```np.max```

In [10]:
%%write_and_run diff_evolution.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import random
from scipy.stats import qmc
import math


def differential_evolution(fobj, bounds, mutation_coefficient=0.5,
                           crossover_coefficient=0.5, population_size=50, iterations=50,
                           init_setting='random', mutation_setting='rand1',
                           selection_setting='current', p_min=0.1, p_max=0.2):
    SEED = 7
    random.seed(SEED)
    np.random.seed(SEED)

    bounds = np.array(bounds)
    dimensions = len(bounds)
    if init_setting == 'LatinHypercube':
        population = qmc.LatinHypercube(d=dimensions, seed=SEED).random(n=population_size)
    elif init_setting == 'Halton':
        population = qmc.Halton(d=dimensions, seed=SEED).random(n=population_size)
    elif init_setting == 'Sobol':
        population = qmc.Sobol(d=dimensions, seed=SEED).random(n=population_size)
    else:
        population = np.random.rand(population_size, dimensions)

    min_bound, max_bound = bounds.T
    diff = np.fabs(min_bound - max_bound)
    population_denorm = min_bound + population * diff
    fitness = np.asarray([fobj(ind) for ind in population_denorm])
    best_idx = np.argmin(fitness)
    best = population_denorm[best_idx]
    for iteration in range(iterations):
        for population_index in range(population_size):
            idxs = np.setdiff1d(np.arange(population_size), [best_idx, population_index], assume_unique=True)
            if mutation_setting == 'rand2':
                a, b, c, d, e = population[np.random.choice(idxs, 5, replace=False)]
                mutant = np.clip(a + mutation_coefficient * (b - c) + mutation_coefficient * (d - e), 0, 1)
            elif mutation_setting == 'best1':
                index_of_best1 = np.argmin(fitness[idxs])

                first, second = min(best_idx, population_index), max(best_idx, population_index)
                if index_of_best1 >= first:
                    index_of_best1 += 1
                if index_of_best1 >= second and first != second:
                    index_of_best1 += 1

                second_best = population[index_of_best1]
                idxs = np.setdiff1d(idxs, [index_of_best1], assume_unique=True)
                b, c = population[np.random.choice(idxs, 2, replace=False)]
                mutant = np.clip(second_best + mutation_coefficient * (b - c), 0, 1)
            elif mutation_setting == 'rand_to_p_best1':
                p = np.random.uniform(p_min, p_max)

                array_of_p_best = fitness[idxs].argsort()[:math.floor(len(fitness[idxs]) * p)]
                index_of_rand_to_p_best1 = np.random.choice(array_of_p_best,1, replace=False)[0]

                first, second = min(best_idx, population_index), max(best_idx, population_index)
                if index_of_rand_to_p_best1 >= first:
                    index_of_rand_to_p_best1 += 1
                if index_of_rand_to_p_best1 >= second and first != second:
                    index_of_rand_to_p_best1 += 1

                second_best = population[index_of_rand_to_p_best1]
                idxs = np.setdiff1d(idxs, [index_of_rand_to_p_best1], assume_unique=True)

                b, c = population[np.random.choice(idxs, 2, replace=False)]
                mutant = np.clip(second_best + mutation_coefficient * (b - c), 0, 1)
            else:
                a, b, c = population[np.random.choice(idxs, 3, replace=False)]
                mutant = np.clip(a + mutation_coefficient * (b - c), 0, 1)
            # Оператор кроссовера
            cross_points = np.random.rand(dimensions) < crossover_coefficient
            if not np.any(cross_points):
                cross_points[np.random.randint(0, dimensions)] = True
            # Рекомбинация (замена мутантными значениями)
            trial = np.where(cross_points, mutant, population[population_index])
            trial_denorm = min_bound + trial * diff
            # Оценка потомка
            result_of_evolution = fobj(trial_denorm)
            # Селекция
            if selection_setting == 'worst':
                selection_index = np.argmax(fitness)
            elif selection_setting == 'random_among_worst':
                for_find = fitness[population_index]
                indexes_of_worse = fitness > for_find
                indexes_of_worse = indexes_of_worse.nonzero()[0]

                if len(indexes_of_worse) == 0:
                    selection_index = population_index
                else:
                    selection_index = np.random.choice(indexes_of_worse, replace=False)
            elif selection_setting == 'random_selection':
                indexes_of_not_best = np.setdiff1d(np.arange(population_size), [best_idx], assume_unique=True)
                selection_index = np.random.choice(indexes_of_not_best, replace=False)
            else:
                selection_index = population_index
            if result_of_evolution < fitness[selection_index]:
                fitness[selection_index] = result_of_evolution
                population[selection_index] = trial
                if result_of_evolution < fitness[best_idx]:
                    best_idx = selection_index
                    best = trial_denorm
        yield best, fitness[best_idx]